In [1]:
using Molly

try
    using GLMakie
catch e
    import Pkg; Pkg.add("GLMakie")
end

In [2]:
n_atomsO = 10
n_atomsH = n_atomsO *2

20

In [3]:
atomO_mass = 16.0u"g/mol"
atomH_mass = 16.0u"g/mol"

16.0 g mol^-1

In [4]:
atomsO = [Atom(mass=atomO_mass, σ=0.3u"nm", ϵ=0.2u"kJ * mol^-1") for i in 1:n_atomsO];
atomsH = [Atom(mass=atomH_mass, σ=0.1u"nm", ϵ=0.2u"kJ * mol^-1") for i in 1:n_atomsH];
atoms = [atomsO;atomsH]; # this concatenates two vectors

In [5]:
# Periodic boundary conditions with a 2 nm cubendary(2.0u"nm") # Periodic boundary conditions with a 2 nm cube
boundary = RectangularBoundary(10.0u"nm", 10.0u"nm") ;

In [6]:
coords = place_atoms(n_atomsO, boundary; min_dist=0.3u"nm"); # Random placement without clashing
for i in 1:length(coords)
    push!(coords, coords[i] .+ [0.1, 0.0]u"nm")
    push!(coords, coords[i] .+ [0.0, 0.1]u"nm")
end

In [7]:
#bonds = InteractionList2Atoms(
#    collect(1:(n_atomsO)),              # First atom indices
#    collect((n_atomsO+1):n_atomsH),     # Second atom indices
#    [HarmonicBond(k=300_000.0u"kJ * mol^-1 * nm^-2", r0=0.1u"nm") for i in 1:(n_atomsO)],
#)

#specific_inter_lists = (bonds,);

In [8]:
fene_k = 250.0u"kJ * mol^-1 * nm^-2"
fene_r0 = 1.6u"nm"
bonds = InteractionList2Atoms(
    collect(1:(n_atomsO)),              # First atom indices
    collect((n_atomsO+1):n_atomsH),     # Second atom indices
    [FENEBond(k=fene_k, r0=fene_r0, σ=1.0u"nm", ϵ=2.5u"kJ * mol^-1") for _ in 1:n_atomsO],)

specific_inter_lists = (bonds,)

(InteractionList2Atoms{Vector{Int64}, Vector{FENEBond{Quantity{Float64, 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, nm^-2, mol^-1), 𝐌 𝐍^-1 𝐓^-2, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}, Quantity{Float64, 𝐋^2 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, mol^-1), 𝐋^2 𝐌 𝐍^-1 𝐓^-2, nothing}}}}}([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20], FENEBond{Quantity{Float64, 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, nm^-2, mol^-1), 𝐌 𝐍^-1 𝐓^-2, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}, Quantity{Float64, 𝐋^2 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, mol^-1), 𝐋^2 𝐌 𝐍^-1 𝐓^-2, nothing}}}[FENEBond{Quantity{Float64, 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, nm^-2, mol^-1), 𝐌 𝐍^-1 𝐓^-2, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}, Quantity{Float64, 𝐋^2 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, mol^-1), 𝐋^2 𝐌 𝐍^-1 𝐓^-2, nothing}}}(250.0 kJ nm^-2 mol^-1, 1.6 nm, 1.0 nm, 2.5 kJ mol^-1), FENEBond{Quantity{Float64, 𝐌 𝐍^-1 𝐓^-2, Unitful.F

In [9]:
temp = 300.0u"K";

In [10]:
velocitiesO = [random_velocity(atomO_mass, temp, dims = 2) for i in 1:n_atomsO];
velocitiesH = [random_velocity(atomH_mass, temp, dims = 2) for i in 1:n_atomsH];
velocities = [velocitiesO; velocitiesH];

In [11]:
#cutoff = DistanceCutoff(1.2u"nm")
pairwise_inters = (LennardJones(),); # Don't forget the trailing comma!

In [12]:
#cutoff = DistanceCutoff(1.2u"nm")

In [13]:
sys = System(
    atoms=atoms,
    coords=coords,
    boundary=boundary,
    velocities=velocities,
    pairwise_inters=pairwise_inters,
    loggers=(
        temp=TemperatureLogger(10),
        coords=CoordinateLogger(1, dims = 2),
    ),
)

System with 30 atoms, boundary RectangularBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[10.0 nm, 10.0 nm])

In [14]:
simulator = VelocityVerlet(
    dt=0.002u"ps",
    coupling=AndersenThermostat(temp, 1.0u"ps"),
)

VelocityVerlet{Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}, AndersenThermostat{Quantity{Float64, 𝚯, Unitful.FreeUnits{(K,), 𝚯, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}}}(0.002 ps, AndersenThermostat{Quantity{Float64, 𝚯, Unitful.FreeUnits{(K,), 𝚯, nothing}}, Quantity{Float64, 𝐓, Unitful.FreeUnits{(ps,), 𝐓, nothing}}}(300.0 K, 1.0 ps), 1)

In [15]:
#simulator = Langevin(
#    dt=0.001u"ps",
#    temperature=temp,
#    friction=1.0u"ps^-1",
#    coupling=MonteCarloBarostat(1.0u"bar", temp, sys.boundary),
#)

In [16]:
simulate!(sys, simulator, 1000)

System with 30 atoms, boundary RectangularBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[10.0 nm, 10.0 nm])

In [17]:
[potential_energy(sys), kinetic_energy(sys)]


2-element Vector{Quantity{Float64, 𝐋^2 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, mol^-1), 𝐋^2 𝐌 𝐍^-1 𝐓^-2, nothing}}}:
   -0.2471967302300884 kJ mol^-1
 2.1835849419344263e32 kJ mol^-1

In [18]:
temperature(sys)

6.037354101202015e32 K

In [19]:
visualize(
    sys.loggers.coords, 
    boundary, 
    "sim_2d.mp4",
    connections=[(1, 2)],
)

"sim_2d.mp4"

In [20]:
display("text/html", """<video autoplay controls><source src="sim_2d.mp4" type="video/mp4"></video>""")